In [1]:
from implementations import *
from helpers import *
import numpy as np
import matplotlib.pyplot as plt

## Load the data ###

In [2]:
data_path='./dataset_to_release/'
train_data_path="./dataset_to_release/x_train.csv"
test_data_path="./dataset_to_release/x_test.csv"

In [3]:
x_train, x_test, y_train, train_ids, test_ids=load_csv_data_all(data_path, sub_sample=False)

## Data Preprocessing

In [42]:
# Handeling the missing values

def replace_nan_by_mean(data):
    ''' function that handels the missing values by replacing them with the column means'''
    nan_indices = np.isnan(data)
    column_means = np.nanmean(data, axis=0)
    data[nan_indices] = np.take(column_means, np.where(nan_indices)[1])
    return data


data_train = replace_nan_by_mean(x_train)


In [5]:
x_train.shape

(328135, 321)

In [6]:
data_train.shape

(328135, 321)

In [7]:
# Data filtering: we only keep relevant features

def filtering(data, data_path):
    columns = extract_first_line(data_path).split(',')
    columns.pop(0)
    columns_to_keep = []
    for c in columns:
        if c.startswith('_'):
            columns_to_keep.append(c)
    indices_to_keep = [columns.index(c) for c in columns_to_keep]
    data_f = data[:, indices_to_keep]
    return(data_f)

In [8]:
features_to_keep = ["_AGE80", "_AGE65YR", "_AGEG5YR", "_AGE_G", "_AIDTST3", "_ASTHMS1", "_BMI5", "_BMI5CAT",
                     "_CASTHM1", "_CHLDCNT", "_CHOLCHK", "_DRDXAR1", "_DRNKWEK", "_DUALCOR", "_DUALUSE"
                     , "_FLSHOT6", "_FRT16", "_FRTLT1", "_FRTRESP", "_FRUITEX", "_FRUTSUM", "_HCVU651",
                       "_LLCPWT", "_LMTACT1", "_LMTSCL1", "_LMTWRK1", "_LTASTH1", "_MICHD", "_MINAC11", "_MINAC21", 
                       "_MISFRTN", "_MISVEGN", "_MRACE1", "_PA30021", "_PA150R2", "_PA300R2", "_PACAT1", "_PAINDX1", 
                       "_PASTAE1", "_PASTRNG", "_PNEUMO2",
                       "_RFBING5", "_RFBMI5", "_RFCHOL", "_RFDRHV5", "_RFHLTH", "_RFHYPE5", "_RFSMOK3", 
                       "_SMOKER3", "_TOTINDA", "_VEG23", "_VEGESUM", "_VEGETEX", "_VEGLT1", "_VEGRESP"]

In [9]:
# Secode version of data filtering, remove 9 more columns
def filtering_2(data,data_path):
    columns = extract_first_line(data_path).split(',')
    columns.pop(0)
    filtered_columns = [col for col in columns if col in features_to_keep]
    indices_to_keep = [columns.index(c) for c in filtered_columns]
    print(len(indices_to_keep))

    data_f = data[:, indices_to_keep]
    return(data_f)

In [10]:
data_train_filtered_2=filtering_2(data_train, train_data_path)

54


In [11]:
data_train_filtered_2.shape


(328135, 54)

In [47]:
# standardization of the data
def standardize(data):
    small_value=1*10**(-9)
    mean=np.mean(data, axis=0)
    std=np.std(data, axis=0)+small_value
    return((data - mean) / (std))

In [13]:
#data_train_standard=standardize(data_train_filtered)

In [14]:
np.sum(y_train==-1)

299160

In [48]:
# feature augmentation
def feature_expansion(data, degree):
    augmented_features=[]
    for i in range(data.shape[1]):
        feature=data[:,i]
        augmented_feature=build_poly(feature, degree)
        augmented_features.append(augmented_feature)

    # Stack the augmented features horizontally
    augmented_data = np.hstack(augmented_features)
    return(augmented_data)

   

## Cross-Validation

In [49]:
def compute_f1_score(true_labels, predicted_labels):
    """
    Computes the F1 score for a classification model using NumPy.

    Parameters:
    true_labels (numpy.ndarray): True labels for the data.
    predicted_labels (numpy.ndarray): Predicted labels from the model.

    Returns:
    f1 (float): The F1 score.
    """
    true_positive = np.sum(np.logical_and(true_labels == 1, predicted_labels == 1))
    false_positive = np.sum(np.logical_and(true_labels == 0, predicted_labels == 1))
    false_negative = np.sum(np.logical_and(true_labels == 1, predicted_labels == 0))
    
    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)
    
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1

In [50]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold.

    Args:
        y:      shape=(N,)
        k_fold: K in K-fold, i.e. the fold num
        seed:   the random seed

    Returns:
        A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold

    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval : (k + 1) * interval] for k in range(k_fold)]
    
    return np.array(k_indices)

In [57]:
def apply_model(test, model):
    pred=(sigmoid(test.dot(model))>=0.6).astype(int)
    return(pred)
    

In [58]:
def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression for a fold corresponding to k_indices

    Args:
        y:          shape=(N,)
        x:          shape=(N,)
        k_indices:  2D array returned by build_k_indices()
        k:          scalar, the k-th fold (N.B.: not to confused with k_fold which is the fold nums)
        lambda_:    scalar, cf. ridge_regression()
        degree:     scalar, cf. build_poly()

    Returns:
        train and test root mean square errors rmse = sqrt(2 mse)

    """

    # ***************************************************
    # get k'th subgroup in test, others in train:
    train_idx=np.reshape(k_indices[[i for i in range(len(k_indices)) if i!=k]], -1)
    test_idx=k_indices[k]

    x_train=x[train_idx,:]
    print(x_train.shape)
    y_train=y[train_idx]
    x_test=x[test_idx,:]
    y_test=y[test_idx]
    
    y_tr=np.expand_dims(y_train, 1)
    y_te=np.expand_dims(y_test, 1)

    y_tr=np.where(y_tr == -1, 0, y_tr)
    print(y_tr, y_tr.shape)
    y_te=np.where(y_te == -1, 0, y_te)

    max_iters = 1000
    gamma=0.5

    # ***************************************************
    # form data with polynomial degree: 
    print('on va auggmenter le data')
    train_data=feature_expansion(x_train, degree)
    test_data=feature_expansion(x_test, degree)
    train_data=standardize(train_data)
    test_data=standardize(test_data)
    # ***************************************************
     # build tx
    tx_tr = np.c_[np.ones((y_train.shape[0], 1)), train_data]
    tx_te = np.c_[np.ones((test_data.shape[0], 1)), test_data]
    print(tx_tr.shape)
    print(tx_te.shape)
    initial_w=np.zeros((tx_tr.shape[1], 1))

    # reg logistic regression: 
    w=reg_logistic_regression(y_tr,tx_tr,lambda_,initial_w, max_iters, gamma)[0]
    print(w.shape)
    print(tx_te.shape)
    y_pred=apply_model(tx_te, w)
    # calculate f1 score on test:
    f1_te=compute_f1_score(y_te, y_pred)
  
    return f1_te

In [59]:
def cross_validation_demo(degree, k_fold, lambdas):
    """cross validation over regularisation parameter lambda.

    Args:
        degree: integer, degree of the polynomial expansion
        k_fold: integer, the number of folds
        lambdas: shape = (p, ) where p is the number of values of lambda to test
    Returns:
        best_lambda : scalar, value of the best lambda
        best_rmse : scalar, the associated root mean squared error for the best lambda
    """

    seed = 12
    #degree = degree
    k_fold = k_fold
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    # define lists to store the loss of training data and test data
    f1_score=np.zeros((len(degree), len(lambdas)))
    # cross validation over lambdas:
    for i in range(len(degree)):
        d=degree[i]
        for j in range(len(lambdas)):
            lambda_=lambdas[j]
            cross_val=[cross_validation(y_train, transformed_data, k_indices, k, lambda_, d) for k in range(k_fold)]
            f1=np.mean(cross_val)
            f1_score[i,j]=f1
    print('on y est presque')
    best_degree=degree[np.unravel_index(np.argmax(f1_score, axis=None), f1_score.shape)[0]]
    best_lambda=lambdas[np.unravel_index(np.argmax(f1_score, axis=None), f1_score.shape)[1]]
    best_f1=np.max(f1_score)
    # print(
    #     "For polynomial expansion up to degree %.f, the choice of lambda which leads to the best test rmse is %.5f with a test rmse of %.3f"
    #     % (degree, best_lambda, f1_score)
    # )
    return best_degree, best_f1 , best_lambda , f1_score

In [60]:
y_train.shape

(328135,)

In [63]:
best_degree, best_f1 , best_lambda , f1_score= cross_validation_demo(np.array([2]).astype(int), 2, np.array([10e-6]))

(164067, 10)
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]] (164067, 1)
on va auggmenter le data
(164067, 31)
(164067, 31)
Current iteration=0, loss=0.6130679430445539
Current iteration=100, loss=0.2922919076002139
Current iteration=200, loss=0.29182737632815947
Current iteration=300, loss=0.29160494557911104
Current iteration=400, loss=0.29146269176436945
Current iteration=500, loss=0.29136471689950866
Current iteration=600, loss=0.29129628325776274
Current iteration=700, loss=0.2912481969723215
Current iteration=800, loss=0.2912141390208778
Current iteration=900, loss=0.2911897281641553
loss=0.291172097334339
(31, 1)
(164067, 31)
(164067, 10)
[[1]
 [0]
 [1]
 ...
 [1]
 [0]
 [0]] (164067, 1)
on va auggmenter le data
(164067, 31)
(164067, 31)
Current iteration=0, loss=0.6135805043550066
Current iteration=100, loss=0.2955131273792419
Current iteration=200, loss=0.29514708924641897
Current iteration=300, loss=0.29497023186263127
Current iteration=400, loss=0.29485862867154655
Current iteration=500, 

/var/folders/v2/nz9bnyf5155f3f5s543g9fhw0000gp/T/ipykernel_47626/1091577108.py:16: RuntimeWarning: invalid value encountered in scalar divide
  precision = true_positive / (true_positive + false_positive)


In [64]:
f1_score

array([[0.]])

In [31]:
best_degree

5

In [49]:
augmented_data=feature_expansion(data_train_filtered_2, 5)

## Training

In [50]:
# standardization of the data
def standardize(data):
    small_value=1*10**(-9)
    mean=np.mean(data, axis=0)
    std=np.std(data, axis=0)+small_value
    return((data - mean) / (std))

In [51]:
data_train_filtered_2

array([[2.00000000e+00, 2.33947890e-01, 2.64741181e+02, ...,
        2.28990981e+00, 2.40679360e+00, 2.00000000e+00],
       [1.00000000e+00, 7.86571559e-01, 1.60771560e+02, ...,
        2.28990981e+00, 2.40679360e+00, 1.96667511e+00],
       [1.00000000e+00, 6.69767282e-01, 5.45089648e+01, ...,
        1.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       ...,
       [1.00000000e+00, 8.07906242e-01, 2.29846589e+02, ...,
        2.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [1.00000000e+00, 7.86571559e-01, 6.42063508e+02, ...,
        2.28990981e+00, 2.40679360e+00, 2.00000000e+00],
       [1.00000000e+00, 7.79554120e-01, 6.84571965e+02, ...,
        2.28990981e+00, 2.40679360e+00, 2.00000000e+00]])

In [52]:
data_standardized = standardize(augmented_data)

In [53]:
data_standardized.shape

(328135, 390)

In [187]:
#split the test set in two


def split_data(x, y, ratio, seed=1):
    """
    split the dataset based on the split ratio. If ratio is 0.8
    you will have 80% of your data set dedicated to training
    and the rest dedicated to testing. If ratio times the number of samples is not round
    you can use np.floor. Also check the documentation for np.random.permutation,
    it could be useful.

    Args:
        x: numpy array of shape (N,), N is the number of samples.
        y: numpy array of shape (N,).
        ratio: scalar in [0,1]
        seed: integer.

    Returns:
        x_tr: numpy array containing the train data.
        x_te: numpy array containing the test data.
        y_tr: numpy array containing the train labels.
        y_te: numpy array containing the test labels.
    """
    N=int(ratio*len(x))
    # set seed
    np.random.seed(seed)
    # split the data based on the given ratio: 
    shuffled_data=np.random.permutation(x)
    #print(shuffled_data)
    np.random.seed(seed)
    x_tr=shuffled_data[:N] #train data
    x_te=shuffled_data[N:] #test data
    y_tr=shuffled_labels[:N]#train labels
    y_te=shuffled_labels[N:]# test labels

    return(x_tr,x_te, y_tr, y_te)

In [188]:
#x_tr,x_te, y_tr, y_te=split_data(data_standardized, y_train, ratio=0.8)


In [189]:
#print(y_tr.shape, x_tr.shape)

(262508, 1) (262508, 276)


In [54]:
# Binary classification using logistic regression

max_iters = 10000
gamma = 0.5

 # build tx
tx_tr = np.c_[np.ones((y_train.shape[0], 1)), data_standardized]
initial_w=np.zeros((tx_tr.shape[1], 1))


In [55]:
y_train=np.expand_dims(y_train, 1)

In [56]:
y_train = np.where(y_train == -1, 0, y_train)

In [57]:
y_train

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [58]:
#grad=calculate_log_likelihood_gradient(y_train,tx_tr,initial_w)


In [59]:
# w=initial_w-gamma*grad
# print(np.min(w), np.max(w))

In [60]:
# loss=calculate_log_likelihood_loss(y_train,tx_tr,initial_w)
# print(loss)

In [51]:
w,loss= logistic_regression(y_train, tx_tr, initial_w, max_iters, gamma=0.5)

Current iteration=0, loss=0.6031202976364645
Current iteration=100, loss=0.25089900822091293
Current iteration=200, loss=0.2493450970050573
Current iteration=300, loss=0.24907393964639502
Current iteration=400, loss=0.24899700173880504
Current iteration=500, loss=0.2489649255881011
Current iteration=600, loss=0.24894547909407405
Current iteration=700, loss=0.24893055173685183
Current iteration=800, loss=0.24891784580236967
Current iteration=900, loss=0.24890658978100963
Current iteration=1000, loss=0.24889644871724068
Current iteration=1100, loss=0.24888723042330485
Current iteration=1200, loss=0.24887880046189664
Current iteration=1300, loss=0.2488710545193124
Current iteration=1400, loss=0.2488639076156132
Current iteration=1500, loss=0.2488572888184579
Current iteration=1600, loss=0.24885113808194817
Current iteration=1700, loss=0.2488454040905189
Current iteration=1800, loss=0.24884004267391904
Current iteration=1900, loss=0.24883501559122753
Current iteration=2000, loss=0.24883028

In [61]:
lambda_ = 10e-4
w_reg,loss_reg= reg_logistic_regression(y_train, tx_tr, lambda_, initial_w, max_iters, gamma)

Current iteration=0, loss=0.6291172121345341
Current iteration=100, loss=0.24737708402551115
Current iteration=200, loss=0.24337991538293094
Current iteration=300, loss=0.24779171148463944
Current iteration=400, loss=0.24213046423714568
Current iteration=500, loss=0.2491880787058022
Current iteration=600, loss=0.24227415696543095
Current iteration=700, loss=0.24801066506027328
Current iteration=800, loss=0.24194129964823735
Current iteration=900, loss=0.24888557069293624
Current iteration=1000, loss=0.24211734755104491
Current iteration=1100, loss=0.24787888264098157
Current iteration=1200, loss=0.24183521074562278
Current iteration=1300, loss=0.24886487760572557
Current iteration=1400, loss=0.2420684023914077
Current iteration=1500, loss=0.24776977394316002
Current iteration=1600, loss=0.2417791215034302
Current iteration=1700, loss=0.24887336007689037
Current iteration=1800, loss=0.24204755752488158
Current iteration=1900, loss=0.2477085792901911
Current iteration=2000, loss=0.241749

In [62]:
w_reg.shape

(391, 1)

## Feature expansion

In [50]:
build_poly(x_train, 4)

NameError: name 'build_poly' is not defined

## Test and Accuracy

In [52]:

#tx_te=np.c_[np.ones((x_test.shape[0], 1)), x_test]

In [53]:
tx_te.shape

(109379, 322)

In [54]:
w.shape

(66, 1)

In [55]:
y_te=np.where(y_te==-1, 0, y_te)

NameError: name 'y_te' is not defined

In [245]:
y_pred=apply_model(tx_te, w)

In [246]:
#Calculate accuracy
correct_predictions = np.sum(y_pred == y_te)
total_samples = len(y_te)
accuracy = correct_predictions / total_samples

In [247]:
compute_f1_score(y_te, y_pred)

0.3508144616607072

In [248]:
print(accuracy)

0.8506102671156689


## Test

In [63]:
xt=replace_nan_by_mean(x_test)

In [64]:
x_test.shape

(109379, 321)

In [65]:
xt_filtered=filtering_2(xt, test_data_path)

65


In [66]:
xt_filtered.shape

(109379, 65)

In [67]:
augmented_data_test=feature_expansion(xt_filtered, 5)

In [68]:
augmented_data_test.shape

(109379, 390)

In [69]:

xt_standardized=standardize(augmented_data_test)
xtest=np.c_[np.ones((xt.shape[0], 1)), xt_standardized]


In [70]:
xtest.shape

(109379, 391)

In [71]:
predictions=apply_model(xtest, w_reg)
predictions=np.where(predictions==0,-1, predictions)

In [72]:
predictions


array([[-1],
       [-1],
       [-1],
       ...,
       [-1],
       [-1],
       [-1]])

In [73]:
create_csv_submission(test_ids, predictions, 'predictions_LR_0.2_5.csv')